In [1]:
# notebooks/eda_modeling.ipynb

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import joblib

C:\Users\hp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\hp\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 1. Load and Understand data

In [68]:
# === Load data ===
train = pd.read_csv('TASK_2/train_set.csv')
test = pd.read_csv('TASK_2/test_set.csv')
B_test = pd.read_csv('TASK_2/blinded_test_set.csv')

In [69]:
train.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238,CLASS
0,ID_1,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,...,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710,0
1,ID_2,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,...,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548,1
2,ID_3,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,...,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704,1
3,ID_4,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,...,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803,0
4,ID_5,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,...,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608,0


In [70]:
B_test.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,ID_101,13249.250000,13323.0,5322.087891,0.401690,0.019253,0.131701,1.965488,0.508780,0.965488,...,453.349939,453.349939,1646.0,1.0,162.029162,0.098438,0.0,30.580378,3.888605,0.098438
1,ID_102,60593.666667,60804.0,21327.521484,0.351976,0.010976,0.042804,2.858719,0.349807,1.858719,...,492.250478,492.250478,7853.0,1.0,961.759455,0.122470,0.0,26.690038,3.695084,0.122470
2,ID_103,51978.833333,52193.0,19574.339844,0.376583,0.010708,0.040742,2.906154,0.344097,1.906154,...,482.387417,482.387417,6644.0,1.0,763.046057,0.114847,0.0,30.037774,3.804517,0.114847
3,ID_104,47737.416667,47943.0,17247.173828,0.361293,0.011891,0.050236,2.710158,0.368982,1.710158,...,475.620243,475.620243,6017.0,1.0,718.741732,0.119452,0.0,27.964103,3.699860,0.119452
4,ID_105,33029.458333,33261.0,15901.136719,0.481423,0.009294,0.030688,3.194060,0.313081,2.194060,...,417.949466,417.949466,4116.0,1.0,314.568513,0.076426,0.0,31.802140,4.078748,0.076426


In [71]:
train.info(), test.info(), B_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Columns: 3240 entries, ID to CLASS
dtypes: float64(3238), int64(1), object(1)
memory usage: 7.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 3240 entries, ID to CLASS
dtypes: float64(3238), int64(1), object(1)
memory usage: 2.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Columns: 3239 entries, ID to Feature_3238
dtypes: float64(3238), object(1)
memory usage: 911.1+ KB


(None, None, None)

In [32]:
train.shape, test.shape, B_test.shape

((315, 3240), (100, 3240), (36, 3239))

In [33]:
type(train), type(test), isinstance(train, pd.DataFrame)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame, True)

In [34]:
#Check which columns are binary features (only contain 0s and 1s).

(train.isin([0, 1]).all()).sum(), (test.isin([0, 1]).all()).sum()
#print(train.columns[train.isin([0, 1]).all()]),print(test.columns[test.isin([0, 1]).all()])

(108, 121)

### 2) DATA CLEANING

In [55]:
# a) Drop Duplicates
train = train.drop_duplicates()
test = test.drop_duplicates()
blind_test= B_test.drop_duplicates()

# b) Missing values
misssing_train=train.isnull().sum().sum()
misssing_test=test.isnull().sum().sum()
misssing_Btest=B_test.isnull().sum().sum()
misssing_train,misssing_test, misssing_Btest

(2668, 1127, 276)

In [56]:
missing_cols = train.columns[train.isnull().any()]
print(missing_cols)

missing_test_cols = test.columns[test.isnull().any()]
print(missing_test_cols)

missing_Btest_cols = blind_test.columns[blind_test.isnull().any()]
print(missing_Btest_cols)

Index(['Feature_1712', 'Feature_1713', 'Feature_1714', 'Feature_1715',
       'Feature_1716', 'Feature_1717', 'Feature_1718', 'Feature_1719',
       'Feature_1720', 'Feature_1721', 'Feature_1722', 'Feature_1723',
       'Feature_1724', 'Feature_1725', 'Feature_1726', 'Feature_1727',
       'Feature_1728', 'Feature_1729', 'Feature_1730', 'Feature_1731',
       'Feature_1732', 'Feature_1733', 'Feature_1734'],
      dtype='object')
Index(['Feature_1712', 'Feature_1713', 'Feature_1714', 'Feature_1715',
       'Feature_1716', 'Feature_1717', 'Feature_1718', 'Feature_1719',
       'Feature_1720', 'Feature_1721', 'Feature_1722', 'Feature_1723',
       'Feature_1724', 'Feature_1725', 'Feature_1726', 'Feature_1727',
       'Feature_1728', 'Feature_1729', 'Feature_1730', 'Feature_1731',
       'Feature_1732', 'Feature_1733', 'Feature_1734'],
      dtype='object')
Index(['Feature_1712', 'Feature_1713', 'Feature_1714', 'Feature_1715',
       'Feature_1716', 'Feature_1717', 'Feature_1718', 'Feature

In [57]:
print(((train.isnull().sum() / len(train)) * 100)[train.isnull().any()])


Feature_1712    36.825397
Feature_1713    36.825397
Feature_1714    36.825397
Feature_1715    36.825397
Feature_1716    36.825397
Feature_1717    36.825397
Feature_1718    36.825397
Feature_1719    36.825397
Feature_1720    36.825397
Feature_1721    36.825397
Feature_1722    36.825397
Feature_1723    36.825397
Feature_1724    36.825397
Feature_1725    36.825397
Feature_1726    36.825397
Feature_1727    36.825397
Feature_1728    36.825397
Feature_1729    36.825397
Feature_1730    36.825397
Feature_1731    36.825397
Feature_1732    36.825397
Feature_1733    36.825397
Feature_1734    36.825397
dtype: float64


In [58]:
# ⚠️ 23 columns have missing values, each with ~36.8% missing.
# This is a significant proportion, and must be handled carefully to avoid bias or loss of information
# If the 23 columns are not strongly correlated with our target or contain low variance, let us consider dropping them using: train.drop(columns=cols_with_high_missing, inplace=True)


# Filter only numerical columns among them (correlation works with numeric data)
missing_numeric_cols = train[missing_cols].select_dtypes(include=[np.number]).columns

#  Compute correlation with target (drop rows with missing values)
correlation_with_target = train[missing_numeric_cols.tolist() + ['CLASS']].dropna().corr()['CLASS'].drop('CLASS')

# Sort by absolute correlation strength
correlation_with_target = correlation_with_target.reindex(correlation_with_target.abs().sort_values(ascending=False).index)

# Display result
print("📊 Correlation of features with missing values vs. CLASS:")
print(correlation_with_target)


📊 Correlation of features with missing values vs. CLASS:
Feature_1712    0.239340
Feature_1725    0.224191
Feature_1726    0.218184
Feature_1730   -0.196653
Feature_1713    0.196653
Feature_1724    0.196653
Feature_1719    0.189340
Feature_1729    0.179577
Feature_1731    0.174440
Feature_1733    0.174440
Feature_1722    0.147537
Feature_1728    0.147537
Feature_1721    0.116677
Feature_1716    0.116677
Feature_1727    0.115881
Feature_1723   -0.076642
Feature_1718    0.076642
Feature_1717    0.076642
Feature_1714   -0.030802
Feature_1715    0.001642
Feature_1720         NaN
Feature_1732         NaN
Feature_1734         NaN
Name: CLASS, dtype: float64


#### The result indicates all features with missing values have only weak correlations with the target class._cols)

#### ❌ Dropping all features with missing values

Although some features showed weak correlations with the target (|r| < 0.25), none demonstrated strong or moderate predictive power. Since missing values can introduce noise, bias, or require complex imputation, and due to the large number of features available,
we choose to remove all columns with missing data to simplify preprocessing and reduce potential overfitting risk.


In [59]:
train = train.drop(columns=missing_cols)
test = test.drop(columns=missing_test_cols)
blind_test = B_test.drop(columns=missing_Btest_cols)

In [60]:
# Separate features and target, and also drop ID since it is an identifier not real features.

# --------------------
X_train = train.drop(columns=['CLASS', 'ID'])
y_train = train['CLASS']

X_test = test.drop(columns=['CLASS','ID'])
y_test = test['CLASS']

X_blind = B_test.drop(columns=['ID'])  # No target assumed

In [61]:
# Again Check for missing values 
print("Missing values:")
print(X_train.isna().sum().sum(), "in train")
print(X_test.isna().sum().sum(), "in test")
print(X_blind.isna().sum().sum(), "in blind test")

Missing values:
0 in train
0 in test
0 in blind test


### c. Outliers

In [62]:
#Clip outliers (by assuming it is safer than removing rows)
# Clip values at 1st and 99th percentiles based on training set

q_low = X_train.quantile(0.01)
q_high = X_train.quantile(0.99)

X_train = X_train.clip(q_low, q_high, axis=1)
X_test = X_test.clip(q_low, q_high, axis=1)
X_blind = X_blind.clip(q_low, q_high, axis=1)



In [63]:
# Standardize (fit only on train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_blind_scaled = scaler.transform(X_blind)


# Output: Scaled data is now ready for modeling

print("Train shape:", X_train_scaled.shape)
print("Test shape:", X_test_scaled.shape)
print("Blind test shape:", X_blind_scaled.shape)

Train shape: (315, 3215)
Test shape: (100, 3215)
Blind test shape: (36, 3215)


In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Helper function to calculate additional metrics
def evaluate_model(y_true, y_pred, y_proba):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)  # Sensitivity (TPR)
    specificity = tn / (tn + fp)  # TNR
    auc = roc_auc_score(y_true, y_proba)

    return {
        "Accuracy": accuracy,
        "F1-Score": f1,
        "Recall (Sensitivity)": recall,
        "Specificity (TNR)": specificity,
        "AUROC": auc
    }

# --------------------------------------
# 1. Logistic Regression with GridSearch
# --------------------------------------
logreg_params = {'C': [0.01, 0.1, 1, 10, 100]}
logreg_grid = GridSearchCV(LogisticRegression(max_iter=1000, solver='liblinear'), logreg_params, cv=5)
logreg_grid.fit(X_train_scaled, y_train)
y_pred_logreg = logreg_grid.predict(X_test_scaled)
y_proba_logreg = logreg_grid.predict_proba(X_test_scaled)[:, 1]

print("\n📘 Logistic Regression Best Params:", logreg_grid.best_params_)
metrics_logreg = evaluate_model(y_test, y_pred_logreg, y_proba_logreg)
print("Metrics:", metrics_logreg)

# --------------------------------------
# 2. SVM with GridSearch (Linear Kernel)
# --------------------------------------
svm_params = {'C': [0.01, 0.1, 1, 10, 100]}
svm_grid = GridSearchCV(SVC(kernel='linear', probability=True), svm_params, cv=5)
svm_grid.fit(X_train_scaled, y_train)
y_pred_svm = svm_grid.predict(X_test_scaled)
y_proba_svm = svm_grid.predict_proba(X_test_scaled)[:, 1]

print("\n📗 SVM Best Params:", svm_grid.best_params_)
metrics_svm = evaluate_model(y_test, y_pred_svm, y_proba_svm)
print("Metrics:", metrics_svm)

# --------------------------------------
# 3. Random Forest with GridSearch
# --------------------------------------
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'max_features': ['sqrt', 'log2']
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5)
rf_grid.fit(X_train_scaled, y_train)
y_pred_rf = rf_grid.predict(X_test_scaled)
y_proba_rf = rf_grid.predict_proba(X_test_scaled)[:, 1]

print("\n📙 Random Forest Best Params:", rf_grid.best_params_)
metrics_rf = evaluate_model(y_test, y_pred_rf, y_proba_rf)
print("Metrics:", metrics_rf)


📘 Logistic Regression Best Params: {'C': 0.1}
Metrics: {'Accuracy': 0.62, 'F1-Score': 0.5, 'Recall (Sensitivity)': 0.4523809523809524, 'Specificity (TNR)': 0.7413793103448276, 'AUROC': 0.6510673234811166}

📗 SVM Best Params: {'C': 0.1}
Metrics: {'Accuracy': 0.58, 'F1-Score': 0.4878048780487805, 'Recall (Sensitivity)': 0.47619047619047616, 'Specificity (TNR)': 0.6551724137931034, 'AUROC': 0.5952380952380952}

📙 Random Forest Best Params: {'max_depth': 10, 'max_features': 'log2', 'n_estimators': 50}
Metrics: {'Accuracy': 0.61, 'F1-Score': 0.41791044776119407, 'Recall (Sensitivity)': 0.3333333333333333, 'Specificity (TNR)': 0.8103448275862069, 'AUROC': 0.6613300492610837}


## Feature Engineering Techniques:

### PCA (Principal Component Analysis)


In [64]:
#📉 PCA (Principal Component Analysis)
from sklearn.decomposition import PCA

# Fit PCA on training data only
pca = PCA(n_components=0.95)  # Retain 95% variance
X_pca_train = pca.fit_transform(X_train_scaled)

# Apply the same transformation to test and blind data
X_pca_test = pca.transform(X_test_scaled)
X_pca_blind = pca.transform(X_blind_scaled)


print("Train shape:", X_pca_train.shape)
print("Test shape:", X_pca_test.shape)
print("Blind test shape:", X_pca_blind.shape)

Train shape: (315, 53)
Test shape: (100, 53)
Blind test shape: (36, 53)


In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Helper function to calculate additional metrics
def evaluate_model(y_true, y_pred, y_proba):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)  # Sensitivity (TPR)
    specificity = tn / (tn + fp)  # TNR
    auc = roc_auc_score(y_true, y_proba)

    return {
        "Accuracy": accuracy,
        "F1-Score": f1,
        "Recall (Sensitivity)": recall,
        "Specificity (TNR)": specificity,
        "AUROC": auc
    }

# --------------------------------------
# 1. Logistic Regression with GridSearch
# --------------------------------------
logreg_params = {'C': [0.01, 0.1, 1, 10, 100]}
logreg_grid = GridSearchCV(LogisticRegression(max_iter=1000, solver='liblinear'), logreg_params, cv=5)
logreg_grid.fit( X_pca_train, y_train)
y_pred_logreg = logreg_grid.predict(X_pca_test)
y_proba_logreg = logreg_grid.predict_proba( X_pca_test)[:, 1]

print("\n📘 Logistic Regression Best Params:", logreg_grid.best_params_)
metrics_logreg = evaluate_model(y_test, y_pred_logreg, y_proba_logreg)
print("Metrics:", metrics_logreg)

# --------------------------------------
# 2. SVM with GridSearch (Linear Kernel)
# --------------------------------------
svm_params = {'C': [0.01, 0.1, 1, 10, 100]}
svm_grid = GridSearchCV(SVC(kernel='linear', probability=True), svm_params, cv=5)
svm_grid.fit(X_pca_train, y_train)
y_pred_svm = svm_grid.predict(X_pca_test)
y_proba_svm = svm_grid.predict_proba(X_pca_test)[:, 1]

print("\n📗 SVM Best Params:", svm_grid.best_params_)
metrics_svm = evaluate_model(y_test, y_pred_svm, y_proba_svm)
print("Metrics:", metrics_svm)

# --------------------------------------
# 3. Random Forest with GridSearch
# --------------------------------------
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'max_features': ['sqrt', 'log2']
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5)
rf_grid.fit(X_pca_train, y_train)
y_pred_rf = rf_grid.predict(X_pca_test)
y_proba_rf = rf_grid.predict_proba(X_pca_test)[:, 1]

print("\n📙 Random Forest Best Params:", rf_grid.best_params_)
metrics_rf = evaluate_model(y_test, y_pred_rf, y_proba_rf)
print("Metrics:", metrics_rf)


📘 Logistic Regression Best Params: {'C': 0.1}
Metrics: {'Accuracy': 0.67, 'F1-Score': 0.5352112676056338, 'Recall (Sensitivity)': 0.4523809523809524, 'Specificity (TNR)': 0.8275862068965517, 'AUROC': 0.6921182266009852}

📗 SVM Best Params: {'C': 0.1}
Metrics: {'Accuracy': 0.59, 'F1-Score': 0.4383561643835616, 'Recall (Sensitivity)': 0.38095238095238093, 'Specificity (TNR)': 0.7413793103448276, 'AUROC': 0.6621510673234811}

📙 Random Forest Best Params: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100}
Metrics: {'Accuracy': 0.57, 'F1-Score': 0.044444444444444446, 'Recall (Sensitivity)': 0.023809523809523808, 'Specificity (TNR)': 0.9655172413793104, 'AUROC': 0.6198686371100164}


In [72]:
# Extract ID columns from original test and blind test
test_ids = test['ID'].values
blind_ids = B_test['ID'].values

# --- Predictions for test data ---
prob_test = logreg_grid.predict_proba(X_pca_test)  # or X_test_selected, X_pca_test as appropriate
df_test_pred = pd.DataFrame(prob_test, columns=['Class_0_Prob', 'Class_1_Prob'])
df_test_pred.insert(0, 'ID', test_ids)
df_test_pred.to_csv("test_predictions.csv", index=False)

# --- Predictions for blind test data ---
prob_blind = logreg_grid.predict_proba(X_pca_blind)  # or X_blind_selected, X_pca_blind
df_blind_pred = pd.DataFrame(prob_blind, columns=['Class_0_Prob', 'Class_1_Prob'])
df_blind_pred.insert(0, 'ID', blind_ids)
df_blind_pred.to_csv("blind_test_predictions.csv", index=False)


### Feature Agglomeration  and Variance Thresholding

In [46]:
#Feature Agglomeration (Clustering-based): Groups similar features into meta-features.
from sklearn.cluster import FeatureAgglomeration
agglo = FeatureAgglomeration(n_clusters=100)
X_train = agglo.fit_transform(X_train)
X_test = agglo.fit_transform(X_test)
X_blind = agglo.fit_transform(X_blind)


print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Blind test shape:", X_blind.shape)

Train shape: (315, 100)
Test shape: (100, 100)
Blind test shape: (36, 100)


In [47]:
# Variance Thresholding: Removes low-variance (uninformative) features.
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.01)
X_train = sel.fit_transform(X_train)
X_test = sel.transform(X_test)
X_blind = sel.transform(X_blind)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Blind test shape:", X_blind.shape)

Train shape: (315, 99)
Test shape: (100, 99)
Blind test shape: (36, 99)


In [48]:

#Standardize (fit only on train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_blind_scaled = scaler.transform(X_blind)


# Output: Scaled data is now ready for modeling

print("Train shape:", X_train_scaled.shape)
print("Test shape:", X_test_scaled.shape)
print("Blind test shape:", X_blind_scaled.shape)

Train shape: (315, 99)
Test shape: (100, 99)
Blind test shape: (36, 99)


In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Helper function to calculate additional metrics
def evaluate_model(y_true, y_pred, y_proba):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)  # Sensitivity (TPR)
    specificity = tn / (tn + fp)  # TNR
    auc = roc_auc_score(y_true, y_proba)

    return {
        "Accuracy": accuracy,
        "F1-Score": f1,
        "Recall (Sensitivity)": recall,
        "Specificity (TNR)": specificity,
        "AUROC": auc
    }

# --------------------------------------
# 1. Logistic Regression with GridSearch
# --------------------------------------
logreg_params = {'C': [0.01, 0.1, 1, 10, 100]}
logreg_grid = GridSearchCV(LogisticRegression(max_iter=1000, solver='liblinear'), logreg_params, cv=5)
logreg_grid.fit( X_train_scaled, y_train)
y_pred_logreg = logreg_grid.predict(X_test_scaled)
y_proba_logreg = logreg_grid.predict_proba( X_test_scaled)[:, 1]

print("\n📘 Logistic Regression Best Params:", logreg_grid.best_params_)
metrics_logreg = evaluate_model(y_test, y_pred_logreg, y_proba_logreg)
print("Metrics:", metrics_logreg)

# --------------------------------------
# 2. SVM with GridSearch (Linear Kernel)
# --------------------------------------
svm_params = {'C': [0.01, 0.1, 1, 10, 100]}
svm_grid = GridSearchCV(SVC(kernel='linear', probability=True), svm_params, cv=5)
svm_grid.fit(X_train_scaled, y_train)
y_pred_svm = svm_grid.predict(X_test_scaled)
y_proba_svm = svm_grid.predict_proba(X_test_scaled)[:, 1]

print("\n📗 SVM Best Params:", svm_grid.best_params_)
metrics_svm = evaluate_model(y_test, y_pred_svm, y_proba_svm)
print("Metrics:", metrics_svm)

# --------------------------------------
# 3. Random Forest with GridSearch
# --------------------------------------
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'max_features': ['sqrt', 'log2']
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5)
rf_grid.fit(X_train_scaled, y_train)
y_pred_rf = rf_grid.predict(X_test_scaled)
y_proba_rf = rf_grid.predict_proba(X_test_scaled)[:, 1]

print("\n📙 Random Forest Best Params:", rf_grid.best_params_)
metrics_rf = evaluate_model(y_test, y_pred_rf, y_proba_rf)
print("Metrics:", metrics_rf)


📘 Logistic Regression Best Params: {'C': 0.01}
Metrics: {'Accuracy': 0.56, 'F1-Score': 0.5686274509803921, 'Recall (Sensitivity)': 0.6904761904761905, 'Specificity (TNR)': 0.46551724137931033, 'AUROC': 0.59688013136289}

📗 SVM Best Params: {'C': 0.01}
Metrics: {'Accuracy': 0.53, 'F1-Score': 0.5346534653465347, 'Recall (Sensitivity)': 0.6428571428571429, 'Specificity (TNR)': 0.4482758620689655, 'AUROC': 0.5736863711001642}

📙 Random Forest Best Params: {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 50}
Metrics: {'Accuracy': 0.42, 'F1-Score': 0.5857142857142857, 'Recall (Sensitivity)': 0.9761904761904762, 'Specificity (TNR)': 0.017241379310344827, 'AUROC': 0.5045155993431856}
